In [14]:
import pandas as pd
import random

# --- 1. Define disaster risk map ---
disaster_risk_map = {
    "Assam": ["Flood", "Earthquake", "Landslide"],
    "Uttarakhand": ["Landslide", "Flood", "Earthquake"],
    "Gujarat": ["Earthquake", "Drought", "Severe Storm"],
    "Tamil Nadu": ["Cyclone", "Flood", "Heatwave"],
    "West Bengal": ["Cyclone", "Flood", "Landslide"],
    "Delhi": ["Heatwave", "Cold Wave"],
    "Rajasthan": ["Drought", "Heatwave"],
    "Bihar": ["Flood", "Heatwave", "Cold Wave"],
    "Maharashtra": ["Drought", "Cyclone", "Severe Storm"],
    "Andhra Pradesh": ["Cyclone", "Flood", "Severe Storm"],
    "Kerala": ["Landslide", "Flood"],
    "Himachal Pradesh": ["Landslide", "Cold Wave"],
    "Odisha": ["Cyclone", "Flood"],
    "Telangana": ["Heatwave", "Drought"],
    "Jharkhand": ["Drought", "Cold Wave"],
    "Punjab": ["Cold Wave", "Flood"],
    "Goa": ["Cyclone", "Flood"]
}

# --- 2. Define checklists for each disaster ---
custom_checklists = {
    "Earthquake": [
        "Secured heavy furniture", "Learned how to turn off utilities", "Emergency contact numbers known",
        "Prepared evacuation plan", "Kept shoes and flashlight beside bed", "First aid kit ready",
        "Important documents accessible", "Practiced earthquake drill", "Know nearest safe zone",
        "Know region's seismic history"
    ],
    "Flood": [
        "Know flood evacuation routes", "Have waterproof bags for documents", "First aid kit ready",
        "Food & water supplies", "Follow flood alerts", "Elevated electrical appliances",
        "Flood insurance taken", "Practiced flood drill", "Emergency contact numbers known",
        "Nearby shelters identified"
    ],
    "Cyclone": [
        "Reinforced windows and doors", "Tree branches trimmed", "Evacuation kit ready",
        "Important documents secured", "Battery-operated radio available", "Emergency contact numbers known",
        "Cyclone alerts followed", "Family trained on safety steps", "Mock drill done", "Water storage prepared"
    ],
    "Drought": [
        "Rainwater harvesting in place", "Water usage optimized", "Stored water for drinking",
        "Drip irrigation used", "Drought-resilient crops selected", "Emergency water plan",
        "Family informed of drought coping methods", "Local drought alerts followed",
        "Check wells and pumps", "Stored food"
    ],
    "Cold Wave": [
        "Warm clothing ready", "Heaters in safe condition", "Insulated home",
        "Emergency contact numbers known", "Backup heating source", "Food & water supplies",
        "Family trained for cold exposure", "Medical needs addressed", "Pets prepared", "Followed cold alerts"
    ],
    "Heatwave": [
        "Hydration plan followed", "Access to cool areas", "Avoided outdoor work during peak hours",
        "First aid for heatstroke known", "Family educated on symptoms", "Fans and AC functional",
        "Windows shaded", "Light cotton clothes used", "Followed heatwave alerts", "Mock drill conducted"
    ],
    "Landslide": [
        "Monitored slope signs", "Retaining walls checked", "Evacuation plan ready",
        "Emergency contact numbers known", "Drainage paths cleared", "Important items secured",
        "Followed weather updates", "Nearby shelters identified", "Practiced landslide drill",
        "Avoided unstable ground"
    ],
    "Severe Storm": [
        "Trimmed trees and shrubs", "Secured outdoor objects", "Emergency kit ready",
        "Listened to storm warnings", "Safe room identified", "Power backups ready",
        "Important papers waterproofed", "Mock drill done", "First aid kit ready", "Emergency contacts updated"
    ],
    "Industrial Hazard": [
        "Know local industry risks", "Toxic leak evacuation plan", "Gas masks and filters ready",
        "Government alerts followed", "Emergency contact numbers known", "Safe routes identified",
        "Community drill participated", "Important documents safe", "Family trained", "Nearby hospitals listed"
    ]
}

# --- 3. Generate synthetic data ---
synthetic_data = []

for state, disaster_types in disaster_risk_map.items():
    for disaster_type in disaster_types:
        for _ in range(100):  # entries per state-disaster combination
            row = {
                "Disaster Type": disaster_type,
                "Location": state,
                "Checklist Type": disaster_type + " Checklist",
                "Household Size": random.randint(1, 10),
                "Disaster Kit Owned": random.randint(0, 1)
            }
            awareness_score = 0
            checklist = custom_checklists.get(disaster_type, [])
            for item in checklist:
                val = random.randint(0, 1)
                row[item] = val
                awareness_score += val
            row["Awareness Score"] = awareness_score
            synthetic_data.append(row)

df_disaster_ready = pd.DataFrame(synthetic_data)

# --- 4. Normalize checklist columns across all rows ---
all_checklist_items = set()
for checklist in custom_checklists.values():
    all_checklist_items.update(checklist)

for item in all_checklist_items:
    if item not in df_disaster_ready:
        df_disaster_ready[item] = 0  # fill missing checklist columns with 0

# --- 5. Preparedness Level Binning ---
df_disaster_ready["Preparedness Level"] = pd.cut(
    df_disaster_ready["Awareness Score"],
    bins=[-1, 3, 7, 10],
    labels=["Needs Urgent Prep", "Moderately Prepared", "Well Prepared"]
)

# --- 6. Derived Feature: Checklist Completion Percentage ---
df_disaster_ready["Checklist Completion %"] = (
    df_disaster_ready[list(all_checklist_items)].sum(axis=1) / 10 * 100
)

# --- 7. Derived Feature: Risk Tier by disaster type ---
risk_tiers = {
    "High": ["Earthquake", "Cyclone", "Flood"],
    "Medium": ["Landslide", "Severe Storm", "Industrial Hazard"],
    "Low": ["Drought", "Heatwave", "Cold Wave"]
}

df_disaster_ready["Risk Tier"] = df_disaster_ready["Disaster Type"].apply(
    lambda x: next((k for k, v in risk_tiers.items() if x in v), "Unknown")
)

# --- 8. Final Checks ---
print("\n✅ Data Generation Summary")
print(f"Total Records: {len(df_disaster_ready)}")
print("\nPreparedness Level Distribution:")
print(df_disaster_ready["Preparedness Level"].value_counts())

# --- 9. Save to CSV ---
df_disaster_ready.to_csv("disaster_preparedness_dataset.csv", index=False)
print("\n📁 Dataset exported as: disaster_preparedness_dataset.csv")



✅ Data Generation Summary
Total Records: 4200

Preparedness Level Distribution:
Preparedness Level
Moderately Prepared    3261
Needs Urgent Prep       719
Well Prepared           220
Name: count, dtype: int64

📁 Dataset exported as: disaster_preparedness_dataset.csv


In [23]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd

# --- 1. Prepare features and target ---
features = [
    'Household Size', 'Disaster Kit Owned', 'Checklist Completion %', 
    'Awareness Score', 'Risk Tier'
]

# Convert 'Risk Tier' into numerical categories
df_disaster_ready['Risk Tier'] = df_disaster_ready['Risk Tier'].map({'High': 2, 'Medium': 1, 'Low': 0})

# Encode the target variable
label_encoder = LabelEncoder()
X = df_disaster_ready[features]
y = label_encoder.fit_transform(df_disaster_ready['Preparedness Level'])  # Encoded to 0, 1, 2

# --- 2. Split the data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 3. Train XGBoost ---
model = XGBClassifier(
    random_state=42,
    eval_metric='mlogloss',
    early_stopping_rounds=10,
    n_estimators=200
)
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)

# --- 4. Evaluate ---
y_pred = model.predict(X_test)
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))  # Show original labels

# --- 5. Save artifacts ---
joblib.dump(model, 'disaster_preparedness_xgb_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')
print("\nModel and label encoder saved.")
# Save feature list for consistent ordering in future predictions
joblib.dump(features, 'model_features.pkl')


[0]	validation_0-mlogloss:0.70285
[1]	validation_0-mlogloss:0.48036
[2]	validation_0-mlogloss:0.33813
[3]	validation_0-mlogloss:0.24206
[4]	validation_0-mlogloss:0.17515
[5]	validation_0-mlogloss:0.12766
[6]	validation_0-mlogloss:0.09353
[7]	validation_0-mlogloss:0.06879
[8]	validation_0-mlogloss:0.05074
[9]	validation_0-mlogloss:0.03752
[10]	validation_0-mlogloss:0.02781
[11]	validation_0-mlogloss:0.02066
[12]	validation_0-mlogloss:0.01539
[13]	validation_0-mlogloss:0.01149
[14]	validation_0-mlogloss:0.00861
[15]	validation_0-mlogloss:0.00647
[16]	validation_0-mlogloss:0.00489
[17]	validation_0-mlogloss:0.00372
[18]	validation_0-mlogloss:0.00284
[19]	validation_0-mlogloss:0.00224
[20]	validation_0-mlogloss:0.00178
[21]	validation_0-mlogloss:0.00143
[22]	validation_0-mlogloss:0.00117
[23]	validation_0-mlogloss:0.00100
[24]	validation_0-mlogloss:0.00088
[25]	validation_0-mlogloss:0.00078
[26]	validation_0-mlogloss:0.00070
[27]	validation_0-mlogloss:0.00066
[28]	validation_0-mlogloss:0.0

['model_features.pkl']